<a href="https://colab.research.google.com/github/yukinaga/min_gen_agent_app/blob/main/section_3/01_agents_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIのAgents SDKを試してみよう！

## 必要パッケージのインストール

In [ ]:
# ▼ Colab で必要なパッケージをインストール
# - openai-agents: OpenAI Agents SDK 本体
# - openai: （一部のホストツールやモデル利用に必要）
# - nest_asyncio: ノートブック上で安全に asyncio（並行処理に必要） を動かすため
!pip -q install --upgrade openai-agents openai nest_asyncio

Agents SDK は openai-agents というパッケージで提供されています。  
まずはこれを入れます。  
SDK のクイックスタートもこの構成（pip install openai-agents）を前提にしています。  
ノートブックで非同期コード（async）を動かすために nest_asyncio を使います（後のセルで適用します）。

## APIキーの設定（Colab Secrets から取得）

In [ ]:
# ▼ Colab の「🔑 Secrets」に事前に OPENAI_API_KEY を保存しておきます
#   （左サイドバー > 🔑 Secrets > + New Secret > Name: OPENAI_API_KEY, Value: sk-...）
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')  # Secrets から取得
if not api_key:
    raise RuntimeError(
        "Colab の Secrets に OPENAI_API_KEY が見つかりません。\n"
        "左サイドバーの🔑 Secretsから 'OPENAI_API_KEY' を追加してください。"
    )

# Agents SDK や OpenAI クライアントが参照できるように環境変数へ設定
os.environ["OPENAI_API_KEY"] = api_key

print("✅ OPENAI_API_KEY を設定しました（値は表示しません）。")

Colab では「Secrets」から安全に読み出して `os.environ` に流し込むのが簡単＆安全です。  

## 最小のエージェントを作って動かす

In [ ]:
# ▼ ノートブック内で async 関数を簡単に実行できるよう準備
import nest_asyncio, asyncio
nest_asyncio.apply()

# ▼ Agents SDK から Agent と Runner を読み込み
from agents import Agent, Runner

# ▼ いちばんシンプルなエージェントを作成
#   - name: エージェントの名前（ログやトレースで識別しやすく）
#   - instructions: モデルへの役割指示（システムプロンプト相当）
simple_agent = Agent(
    name="Friendly Tutor",
    instructions="あなたは親切な家庭教師です。できるだけ分かりやすく丁寧に日本語で説明してください。",
)

# ▼ ユーザーからの質問に答えさせる
async def main():
    # Runner.run(エージェント, ユーザー入力文字列)
    result = await Runner.run(simple_agent, "微分って何ですか？高校生にも分かるように説明して。")
    print("=== エージェントの最終出力 ===")
    print(result.final_output)           # 生成された最終回答テキスト
    # print(result) とすると、ツール呼び出し等の詳細も含むリッチな結果が見られます

# ノートブックでも動くように run_until_complete を利用
asyncio.get_event_loop().run_until_complete(main())

Agents SDK の 基本構成 は「Agent（モデル+手順+ツール）」「Runner（実行器）」。  
まずはツールなしで会話してみます。  
  
結果オブジェクトの `final_output` が最終回答です。  
詳細は SDK の「Results」や「Tracing」で確認できます（後述）。

## 独自ツール（関数ツール）を足す：電卓ツールの例

In [ ]:
# ▼ Function Tool を使って、エージェントが Python 関数を呼べるようにします
from agents import function_tool, Agent, Runner

@function_tool
def calc(op: str, a: float, b: float) -> float:
    """
    かんたんな電卓ツール。
    Args:
      op: "add" | "sub" | "mul" | "div"
      a: 1つ目の数
      b: 2つ目の数
    Returns:
      演算結果の数値
    """
    if op == "add":
        return a + b
    if op == "sub":
        return a - b
    if op == "mul":
        return a * b
    if op == "div":
        return a / b
    raise ValueError("op は add/sub/mul/div のいずれかにしてください")

tool_agent = Agent(
    name="Calc Assistant",
    instructions=(
        "あなたは質問文から必要なら calc ツールを呼び出し、"
        "計算結果を日本語で分かりやすく説明します。"
        "ツールを使った場合は、途中式や根拠も簡潔に示してください。"
    ),
    tools=[calc],  # ← 独自ツールを渡す
)

import nest_asyncio, asyncio
nest_asyncio.apply()

async def main():
    q = "半径が 3 の円の面積はいくつ？（π=3.14159で計算して）"
    # 面積 = π * r^2 を求めるため、モデルは calc(op='mul', ...) などを自動で選ぶはずです
    result = await Runner.run(tool_agent, q)
    print("=== エージェントの最終出力 ===")
    print(result.final_output)

asyncio.get_event_loop().run_until_complete(main())

**Function Tool** は任意の Python 関数をそのまま「ツール」として公開できます。  
デコレーター `@function_tool` を付けるだけで、引数スキーマや説明文を自動抽出してくれます。  
AIモデルは会話の流れから「ツールを使うべきか」を自動判断して、必要なときだけ calc を呼び出します（関数の実行はあなたのコード側で行われます）。

## 2エージェントの「振り分け（ハンドオフ）」を体験

In [ ]:
# ▼ 質問の内容に応じて、適切な専門家エージェントへ自動ハンドオフさせます
from agents import Agent, Runner

history_tutor = Agent(
    name="History Tutor",
    handoff_description="歴史に関する質問を専門に扱うエージェント",
    instructions="歴史的な出来事や人物を、背景も含めて丁寧に説明してください。"
)

math_tutor = Agent(
    name="Math Tutor",
    handoff_description="数学に関する質問を専門に扱うエージェント",
    instructions="数式や例を使って、段階的にやさしく解説してください。"
)

triage = Agent(
    name="Triage Agent",
    instructions="ユーザーの質問内容を見て、history_tutor または math_tutor のどちらに渡すか判断します。",
    handoffs=[history_tutor, math_tutor],  # ← 候補先を列挙
)

import nest_asyncio, asyncio
nest_asyncio.apply()

async def main():
    # 歴史寄りの質問
    r1 = await Runner.run(triage, "織田信長が日本史に与えた影響を教えて")
    print("【歴史寄りの質問】\n", r1.final_output, "\n")

    # 数学寄りの質問
    r2 = await Runner.run(triage, "微分と積分の違いを、高校生向けに例を出して説明して")
    print("【数学寄りの質問】\n", r2.final_output)

asyncio.get_event_loop().run_until_complete(main())

Agents SDK は「複数エージェントのオーケストレーション」を簡単に扱えます。  
上の例では Triage（振り分け） エージェントが、質問内容を見て専門家へ ハンドオフ します。  
大きなアプリでは、さらにガードレール（不適切入力の遮断）やトレース（実行の可視化）を組み合わせます。  

## ホストツールを使う：Web 検索の付与

In [ ]:
# ▼ Web 検索ツールを付けたエージェント（権限があるアカウントで利用可能）
from agents import Agent, Runner, WebSearchTool

web_agent = Agent(
    name="Web Helper",
    instructions="質問に対して、最新の公的情報・一次情報にあたって回答します。根拠リンクも列挙してください。",
    tools=[WebSearchTool()],  # ← これだけで Web 検索が使えるようになります
)

import nest_asyncio, asyncio
nest_asyncio.apply()

async def main():
    q = "直近のノーベル賞（物理学賞）の受賞テーマを教えて。要点を3つに整理して。"
    result = await Runner.run(web_agent, q)
    print(result.final_output)   # 検索の要約＋根拠を期待

asyncio.get_event_loop().run_until_complete(main())

「Hosted tools」は WebSearch / FileSearch / Code Interpreter / Computer Use / 画像生成 / Hosted MCP などがあり、`Agent(..., tools=[ ... ])` へ追加するだけで利用できます。  
実行は OpenAI 側で自動処理され、結果が会話へ反映されます。

## 実行の可視化（トレース）を見る
実行の流れ（どのエージェントが使われ、どのツールが呼ばれたか等）は OpenAI ダッシュボードの「Logs/Traces」 で確認できます。開発・デバッグに便利です。